In [2]:
import pandas as pd
from geopy.distance import geodesic

In [3]:
route_short_name = '510'
input = 'data_in'
output = 'data_out'
static_data = 'mdb-2253'

rt_trips = pd.read_csv(output + '/' + route_short_name + '-rt.csv', sep=',')
# print(rt_trips)

static_trips = pd.read_csv(output + '/' + route_short_name + '-static.csv')
# print(static_trips)

stops = pd.read_csv(input + '/' + static_data + '/stops.txt', sep=',')
# print(stops)

stop_times = pd.read_csv(input + '/' + static_data + '/stop_times.txt', sep=',')
# print(stop_times)

FileNotFoundError: [Errno 2] No such file or directory: 'data_out/510-rt.csv'

In [ ]:
# Merge rt info with static info
rt_static_merged_trips = rt_trips.merge(static_trips, on='trip_id', how='outer')
rt_static_merged_trips = rt_static_merged_trips.drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'route_id'], axis=1).dropna()
rt_static_merged_trips.reset_index(drop = True, inplace = True)
rt_static_merged_trips.to_csv(output + '/' + route_short_name + '-rt-static.csv')
# print(rt_static_merged_trips)

In [ ]:
# Merge stop_times with stops
stop_info = stop_times.merge(stops, on="stop_id")
stop_info = stop_info[["trip_id","arrival_time","departure_time",
                 "stop_id","stop_sequence","stop_lat", "stop_lon", "stop_name"]]
# print(stop_info)

In [ ]:
# Merge stop_info and rt_static_trips on trip_id
merged_temp = stop_info.merge(rt_static_merged_trips, on="trip_id", suffixes=("_trip", "_rt"))
merged = merged_temp[["trip_id", "now", "arrival_time_trip","departure_time_trip",
                 "stop_sequence_trip", "stop_sequence_rt",
                 "stop_lat_trip", "stop_lon_trip", "stop_name_trip",
                 "vehicle_latitude", "vehicle_longitude"]]
# print(merged)

# Filter rows where stop_sequence is one less or one more
filtered = merged[
    (merged["stop_sequence_rt"] == merged["stop_sequence_trip"] - 1) |
    (merged["stop_sequence_rt"] == merged["stop_sequence_trip"] + 1) |
    (merged["stop_sequence_rt"] == merged["stop_sequence_trip"])
]
filtered_data = filtered[["trip_id", "stop_sequence_trip", "now", "arrival_time_trip", "departure_time_trip", "stop_name_trip", "stop_lat_trip", "stop_lon_trip", "vehicle_latitude", "vehicle_longitude"]]
# filtered_data.to_csv(output + '/' + 'filtered_data.csv')
# print(filtered_data)

In [ ]:
# Calculate distance to each stop
def find_distance_to_stop(group):
    group['distance_to_stop'] = group.apply(
        lambda row: geodesic(
            (row['vehicle_latitude'], row['vehicle_longitude']),
            (row['stop_lat_trip'], row['stop_lon_trip'])
        ).meters,
        axis=1
    )
    return group

calculated_distance = filtered_data.groupby("trip_id").apply(find_distance_to_stop)
# print(calculated_distance)
calculated_distance.reset_index(drop = True, inplace = True)
# calculated_distance.to_csv(output + '/' + 'calculated_distance.csv')


/tmp/ipykernel_128601/1948330709.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  calculated_distance = filtered_data.groupby("trip_id").apply(find_distance_to_stop)


In [ ]:
# Identify which stop is the closest (by stop_sequence)
calculated_distance['closest_stop_sequence'] = (
    calculated_distance.groupby('trip_id')['distance_to_stop']
    .transform(lambda x: calculated_distance.loc[x.idxmin(), 'stop_sequence_trip'])
)
# print(calculated_distance)
calculated_distance.reset_index(drop = True, inplace = True)
calculated_distance.to_csv(output + '/' + 'calculated_distance.csv')

In [ ]:
rt_static_merged_trips_concise = rt_static_merged_trips[['trip_id', 'stop_sequence']]
calculated_distance_concise = calculated_distance[['trip_id', 'closest_stop_sequence']]
comparison = (calculated_distance_concise.merge(rt_static_merged_trips_concise, on='trip_id', how='right'))[['trip_id', 'stop_sequence', 'closest_stop_sequence']]

# Evaluate whether bus is on/ahead/behind schedule
result = comparison.drop_duplicates()
result['status'] = result.apply(
    lambda row: 
        'on' if row['closest_stop_sequence'] == row['stop_sequence'] 
        else
        'ahead' if row['closest_stop_sequence'] > row['stop_sequence'] 
        else
        'behind',
    axis=1
)
result.reset_index(drop = True, inplace = True)
result.to_csv(output + '/' + 'result.csv')
# print(result)

/tmp/ipykernel_128601/2030282519.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['status'] = result.apply(
